In [46]:
import numpy as np
import pandas as pd
import os
import re

# === 1. Transformation & Adjustment ===

def calculate_transformation_coeffs(src_points: np.ndarray, tgt_points: np.ndarray) -> tuple:
    X_src, Y_src = src_points[:, 0], src_points[:, 1]
    X_tgt, Y_tgt = tgt_points[:, 0], tgt_points[:, 1]

    design_matrix = np.vstack([np.ones_like(X_src), X_src, Y_src]).T
    A_coeffs, _, _, _ = np.linalg.lstsq(design_matrix, X_tgt, rcond=None)
    B_coeffs, _, _, _ = np.linalg.lstsq(design_matrix, Y_tgt, rcond=None)

    return (*A_coeffs, *B_coeffs)

def apply_transformation(src_points: np.ndarray, A0: float, A1: float, A2: float, B0: float, B1: float, B2: float) -> np.ndarray:
    X, Y, Z = src_points[:, 0], src_points[:, 1], src_points[:, 2]
    Xp = A0 + A1 * X + A2 * Y
    Yp = B0 + B1 * X + B2 * Y
    Zp = Z  # Z remains unchanged
    return np.stack((Xp, Yp, Zp), axis=1)

# === 2. Reporting ===

def format_transformation_report(A0, A1, A2, B0, B1, B2, src_points, tgt_points, residuals, z_warning=False) -> str:
    report = []

    if z_warning:
        report.append("\nWARNING: Z and Z' are not equal. Consider using a 3D affine transformation\n.")

    report += [
        "Linear Affine Transformation (2D)",
        "---------------------------------",
        "This transformation maps source coordinates (X, Y) to target coordinates (X', Y') using:\n",
        "X' = A0 + A1 * X + A2 * Y + A0",
        "Y' = B0 + B1 * X + B2 * Y\n",
        "Transformation Coefficients:",
        f"A0 = {A0:.12f}", f"A1 = {A1:.12f}", f"A2 = {A2:.12f}\n",
        f"B0 = {B0:.12f}", f"B1 = {B1:.12f}", f"B2 = {B2:.12f}\n",
        "Known point pairs",
        "------------------------------------------------"
    ]

    for i, (src, tgt) in enumerate(zip(src_points, tgt_points), 1):
        report.append(f"{i:<9} {src[0]:>10.3f}  {src[1]:>10.3f}  {src[2]:>6.3f}")
        report.append(f"      =>  {tgt[0]:>10.3f}  {tgt[1]:>10.3f}  {tgt[2]:>6.3f}")

    report += [
        "\nResiduals",
        "Number          dX        dY        dZ",
        "------------------------------------------------"
    ]

    dX, dY, dZ = residuals[:, 0], residuals[:, 1], residuals[:, 2]
    for i, (dx, dy, dz) in enumerate(residuals, 1):
        report.append(f"{i:<16}{dx:+9.4f}{dy:+10.4f}{dz:+10.4f}")

    avg_mag = np.mean(np.linalg.norm(residuals, axis=1))
    rms = np.sqrt(np.mean(residuals**2))

    report += [
        "------------------------------------------------",
        f"Max\t{np.max(np.abs(dX)):+9.4f}{np.max(np.abs(dY)):>10.4f}{np.max(np.abs(dZ)):>10.4f}",
        f"Avg\t{avg_mag:10.4f}  {avg_mag:8.4f}  {avg_mag:8.4f}",
        f"RMS\t{rms:10.4f}  {rms:8.4f}  {rms:8.4f}"
    ]

    return "\n".join(report)

# === 3. I/O Utilities ===

def load_point_file(file_path: str, label: str) -> np.ndarray:
    try:
        # Try reading with header first
        df = pd.read_csv(file_path)

        # If the first row is not numeric, treat it as a header
        if not np.issubdtype(df.iloc[0, 0], np.number):
            df = pd.read_csv(file_path, header=1)

        if df.empty:
            raise ValueError(f"{label} file is empty.")
        if df.shape[1] < 3:
            raise ValueError(f"{label} file must have at least 3 columns (X, Y, Z).")

        # Extract the first 3 columns, regardless of name
        coords = df.iloc[:, :3].to_numpy()

        if not np.issubdtype(coords.dtype, np.number):
            raise ValueError(f"{label} file must contain numeric values in the first three columns.")

        return coords

    except Exception as e:
        raise FileNotFoundError(f"Error loading {label} file: {e}")


def write_report(report_text: str, directory: str) -> None:
    report_path = os.path.join(directory, "2d_affine_transformation_report.txt")
    with open(report_path, "w") as f:
        f.write(report_text)
    print(f"\nReport written to: {report_path}")


# === 5. Affine Estimation & Reporting Pipeline ===

def estimate_affine_transformation(src_points: np.ndarray, tgt_points: np.ndarray) -> dict:
    """Estimate affine transformation coefficients and residuals."""
    A0, A1, A2, B0, B1, B2 = calculate_transformation_coeffs(src_points, tgt_points)
    transformed = apply_transformation(src_points, A0, A1, A2, B0, B1, B2)
    residuals = tgt_points - transformed
    z_warning = not np.allclose(src_points[:, 2], tgt_points[:, 2], atol=1e-6)

    return {
        "A0": A0, "A1": A1, "A2": A2,
        "B0": B0, "B1": B1, "B2": B2,
        "residuals": residuals,
        "z_warning": z_warning,
        "transformed": transformed
    }


def write_affine_report(data_dir: str, src_points: np.ndarray, tgt_points: np.ndarray, results: dict) -> None:
    """Format and write the affine transformation report to disk."""
    report_text = format_transformation_report(
        results["A0"], results["A1"], results["A2"],
        results["B0"], results["B1"], results["B2"],
        src_points, tgt_points, results["residuals"],
        results["z_warning"]
    )
    write_report(report_text, data_dir)


def run_affine_transformation_pipeline(data_dir: str, src_points_file: str, tgt_points_file: str) -> None:
    """Pipeline to load data, estimate transformation, and write report."""

    # Load input data
    src_path = os.path.join(data_dir, src_points_file)
    tgt_path = os.path.join(data_dir, tgt_points_file)
    src_points = load_point_file(src_path, "source")
    tgt_points = load_point_file(tgt_path, "target")

    # Print loaded coordinates
    print("\nSource Points:")
    print(src_points)

    print("\nTarget Points:")
    print(tgt_points)

    # Estimate transformation
    results = estimate_affine_transformation(src_points, tgt_points)

    # Optional warning
    if results["z_warning"]:
        print("\nWARNING: Z and Z' are not equal. Consider using a 3D affine transformation.")
    
    # Write report
    write_affine_report(data_dir, src_points, tgt_points, results)


In [47]:
DIRECTORY = r"C:\Users\USFJ139860\WSP O365\Southwest Geomatics - Business Development\Marketing & Presentations\2025\NMDOT CIVIL3D CUSTOM COORDINATE SYSTEMS\SMART1-9\CSV\XYZ_GPS"
SOURCE_POINTS = "NM128_SMART1-9  -TM-SF XYZ SOURCE.csv"
TARGET_POINTS = "NM128_SMART1-9 XYZ TARGET.csv"

run_affine_transformation_pipeline(DIRECTORY, SOURCE_POINTS, TARGET_POINTS)


Source Points:
[[644005.776    488591.5266     3026.472304]
 [650091.4286   489757.2155     3002.860299]
 [657073.5145   490392.2533     2985.52682 ]
 [662336.8505   491852.4517     2999.88517 ]
 [666244.1401   486785.4095     3002.880065]
 [675285.0151   485580.3277     3015.77126 ]
 [682155.9446   484719.7404     3126.098688]
 [689869.8368   483847.3137     3216.510338]
 [696855.1269   482408.7643     3281.081365]]

Target Points:
[[643999.1308   488507.6308     3026.472217]
 [650080.3439   489696.2621     3002.860233]
 [657059.9853   490357.627      2985.526774]
 [662317.777    491837.6646     2999.885144]
 [666244.1481   486785.394      3002.880022]
 [675289.5036   485614.4166     3015.771232]
 [682163.6297   484779.7478     3126.098671]
 [689880.7573   483936.4186     3216.510335]
 [696871.4229   482524.2231     3281.081371]]

Report written to: C:\Users\USFJ139860\WSP O365\Southwest Geomatics - Business Development\Marketing & Presentations\2025\NMDOT CIVIL3D CUSTOM COORDINATE S